# Knowledge Distillation For Fine-Tuning A GPT-3.5 Judge

There has been recent research that demonstrated GPT-4's ability to closely align to human judges when evaluating LLM generated texts (e.g., see [[1]](https://arxiv.org/abs/2306.05685), [[2]](https://arxiv.org/abs/2303.16634)). In this notebook, we demonstrate how to use the `llama_index` library to distill knowledge from GPT-4 to GPT-3.5 so that the smaller GPT-3.5 becomes closer to GPT-4 performance; and by proxy, closer to human judges.

To do so, we take the following steps:

1. Generate datasets: `train` and `test`
2. Perform knowledge distillation (using `train`)
3. Evaluate the distilled model  on `test`

## 0 Prompt Templates & Asyncio Event Loop

In [ ]:
PROMPTS = {
    "QUESTION_GEN": (
        "You are a Teacher/ Professor. Your task is to setup "
        "a quiz/examination. Using the provided context, formulate "
        "a single question that captures an important fact from the "
        "context. Restrict the question to the context information provided."
    )
}

## 1 Generate datasets: `train` and `test`

We should not lose sight on the ultimate goal here, which is to build an LLM judge that closely matches to human judges when evaluating LLM-generated texts. The work we need to do in this step, therefore, is to build a set of generated texts that our LLM judges will judge. More specifically, we will follow the "pairwise comparison" evaluation design pattern, where one text generation is passed to an LLM judge that is subsequently prompted to assign a score between 0 and 1 (higher is better).

To generate a varied set of texts we'll use the following LLM text-generators:
1. HuggingFace: Llama2-7B (chat)
2. HuggingFace: Mistral-7B (instruct)
3. HuggingFace: Falcon-7B (instruct)

The generation task we ask of each of these models will be to generate an abstractive answer to question when provided relevant context (i.e., RAG).

### Using `DatasetGenerator` to build `train` and `test`

The specific procedure we will use here involves generating questions against a set of chunks of a given `Document`. With the `<question, chunk>` pairs in hand, (for which we can merely treat as a "simulated" retrieval), we pass this information to the three LLM generators and prompt them each to generate an answer.

Hang tight, we're almost there (sort of). Since we want to distill GPT-4 abilities for this task to GPT-3.5, we now need to generate GPT-4 judgements on the generated answers. To do this, we will pass the `<question, answer A, answer B>` (where `A` and `B` represent answers from any two of the LLM text-generators) as context to the GPT-4 judge and prompt it to decide the better answer of the two.

With all of that we can now build a `dataset` that looks like the one below.
| question | context-answer-A-answer-B | gpt-4-evaluation |
|----------|---------------------------|------------------|
| ...      | ...                       | ...              |

And finally, to get `train` and `test` we will simply randomly shuffle `dataset` and split it using a 70/30 ratio. (Phew!)

#### Generate Questions and LLM-Generated Answers

With all that out of the way, let's spring into action. First, we will download the reference pdf document and create the set of questions against it.

In [ ]:
# Download the pdf document — Uncomment the line of code below
# !curl https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf --output IPCC_AR6_WGII_Chapter03.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20.7M  100 20.7M    0     0   619k      0  0:00:34  0:00:34 --:--:--  648k  441k      0  0:00:48  0:00:02  0:00:46  441k     0   611k      0  0:00:34  0:00:24  0:00:10  635k616k      0  0:00:34  0:00:32  0:00:02  632k


In [ ]:
import random
from llama_index import SimpleDirectoryReader, ServiceContext

# load a document
documents = SimpleDirectoryReader(
    input_files=["paul_graham_essay.txt"]
).load_data()

# Shuffle the documents
random.seed(42)
random.shuffle(documents)

In [ ]:
# generate questions against chunks
from llama_index.evaluation import DatasetGenerator
from llama_index.llms import OpenAI

# set context for llm provider
gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3)
)

# instantiate a DatasetGenerator
dataset_generator = DatasetGenerator.from_documents(
    documents,
    question_gen_query=PROMPTS["QUESTION_GEN"],
    service_context=gpt_35_context,
    num_questions_per_chunk=50,
)

In [ ]:
# use DatasetGenerator to create questions from nodes
questions = dataset_generator.generate_questions_from_nodes(num=10)

In [ ]:
# let's take a look at a few of these
for q in questions[:5]:
    print(q)

Question: What was the author's initial experience with programming on the IBM 1401?
What language was regarded as the language of AI during the time the author was learning AI at Cornell?
Question: What prompted the author to consider pursuing a career in art?
What was the author's experience like studying at the Accademia in Florence?
What important lesson did the author learn from their experience at Interleaf?


Now that we have the questions, the next step is to generate answers using the three LLM text-generators: Llama-2, Mistral, and Falcon.

In [ ]:
# Create vector index
from llama_index import VectorStoreIndex
from llama_index.indices.vector_store.retrievers import VectorIndexRetriever

index = VectorStoreIndex.from_documents(documents=documents)

retriever = VectorIndexRetriever(  # what embeddings are being used?
    index=index,
    node_ids=list(index.index_struct.nodes_dict.values()),
    similarity_top_k=2,
)

In [ ]:
import os

HUGGING_FACE_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

In [ ]:
from llama_index.query_engine.retriever_query_engine import (
    RetrieverQueryEngine,
)
from llama_index.llms import HuggingFaceInferenceAPI
from llama_index.llm_predictor import LLMPredictor


def create_query_engine(hf_name: str) -> RetrieverQueryEngine:
    """Create a RetrieverQueryEngine using the HuggingFaceInferenceAPI LLM"""
    if hf_name not in hf_llm_generators:
        raise KeyError("model not listed in hf_llm_generators")
    llm = HuggingFaceInferenceAPI(
        model_name=hf_llm_generators[hf_name],
        context_window=2048,  # to use refine
        token=HUGGING_FACE_TOKEN,
    )
    context = ServiceContext.from_defaults(llm_predictor=LLMPredictor(llm=llm))
    return RetrieverQueryEngine.from_args(
        retriever=retriever, service_context=context
    )

In [ ]:
# define our llm-generators
hf_llm_generators = {
    "mistral-7b-instruct": "mistralai/Mistral-7B-Instruct-v0.1",
    "llama2-7b-chat": "meta-llama/Llama-2-7b-chat-hf",
    "falcon-7b-instruct": "tiiuae/falcon-7b-instruct",
}

query_engines = {
    mdl: create_query_engine(mdl) for mdl in hf_llm_generators.keys()
}

/Users/nerdai/Library/Caches/pypoetry/virtualenvs/llama-index-e6cjsBOJ-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
query_engines

{'mistral-7b-instruct': <llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x17fe05000>,
 'llama2-7b-chat': <llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x17fe04820>,
 'falcon-7b-instruct': <llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x17fe07160>}

Now, we can create the master dataset

In [ ]:
import tqdm

dataset = []
for q in tqdm.tqdm(questions):
    data_entry = {"question": q}

    responses = {}
    for name, engine in query_engines.items():
        responses[name] = str(engine.query(q))

    data_entry["answers"] = responses
    dataset.append(data_entry)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [03:41<00:00, 22.17s/it]


In [ ]:
dataset

[{'question': "Question: What was the author's initial experience with programming on the IBM 1401?",
  'answers': {'mistral-7b-instruct': "\n\nThe author's initial experience with programming on the IBM 1401 was that he couldn't figure out what to do with it. He couldn't understand the language he was using and couldn't think of any interesting problems to solve. He was puzzled by the machine and its capabilities. However, he was able to salvage some of the wreckage of his plans and focus on Lisp, which he found interesting for its own sake and not just for its association with AI. He decided to write a book about Lisp hacking, which helped him learn the language better. He was drawn to systems work, but he realized that any program he wrote would be obsolete in a couple of decades. He briefly considered using surplus Xerox Dandelions, but they were too slow by present standards.",
   'llama2-7b-chat': "\nThe author's initial experience with programming on the IBM 1401 was not memorab

In [ ]:
# save these generations for future use
import json

with open("qa_dataset.jsonl", "w") as outfile:
    for entry in dataset:
        print(json.dumps(entry), file=outfile)

#### Generate GPT-4 Evaluations

In [ ]:
from llama_index.llms import OpenAI

In [ ]:
# # for loading the jsonl file
# with open('dataset.jsonl') as f:
#     loaded_data = [json.loads(line) for line in f]

In [ ]:
# create our dataset, and split into train and test

## 2 Perform knowledge distillation

Okay, it's now time to distill some knowledge from GPT-4 to GPT-3.5 To do this, we will make use of `OpenAIFinetuneEngine` class of `llama_index`. 